# Loan Status Prediction (edit title)

## Introduction

#### Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal

When an individual requests a loan from an entity such as a government, corporation or bank, they must submit an application in which they request the amount they need to borrow. The entity can approve or deny the request, based on factors such as the individual’s financial history and character. 

#### Clearly state the question you will try to answer with your project

In our project, we are trying to predict whether or not a person will have their loan approved based on the characteristics described in the data set. The predictive question we are trying to answer in our project is “Will a loan applicant with [insert variables] have their loan approved or denied?”. 

#### Identify and describe the dataset that will be used to answer the question

The data set provided by Kaggle is focused on providing information on whether or not an individual who applied for a property loan receives the loan. The data set has 13 columns and includes data on many characteristics of the applicant, including categorical variables such as gender, marital status, number of dependents, education, whether or not they’re self-employed, whether or not their credit history meets guidelines, property area, and loan status, as well as quantitative variables such as income, coapplicant income, loan amount, and loan term. 

## Preliminary exploratory data analysis

In [8]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('tests.R')
source('cleanup.R')

Warning message in file(filename, "r", encoding = encoding):
“cannot open file 'tests.R': No such file or directory”


ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [9]:
# reading the data

loan_data <- read_csv("Group Project/loan_data.csv")
loan_data

Rows: 381 Columns: 13
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (8): Loan_ID, Gender, Married, Dependents, Education, Self_Employed, Pro...
dbl (5): ApplicantIncome, CoapplicantIncome, LoanAmount, Loan_Amount_Term, C...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>
LP001003,Male,Yes,1,Graduate,No,4583,1508,128,360,1,Rural,N
LP001005,Male,Yes,0,Graduate,Yes,3000,0,66,360,1,Urban,Y
LP001006,Male,Yes,0,Not Graduate,No,2583,2358,120,360,1,Urban,Y
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
LP002978,Female,No,0,Graduate,No,2900,0,71,360,1,Rural,Y
LP002979,Male,Yes,3+,Graduate,No,4106,0,40,180,1,Rural,Y
LP002990,Female,No,0,Graduate,Yes,4583,0,133,360,0,Semiurban,N


In [5]:
# splitting between training and testing sets 

loan_split <- initial_split(loan_data, prop = 0.75, strata = Loan_Status) 
loan_training <- training(loan_split) 
loan_testing <- testing(loan_split) 

# exploratory data analysis (table) 

loan_counts <- loan_training |>
    group_by(Loan_Status) |>
    summarise(Number_of_Observations = n())
loan_counts

loan_missing_data <- loan_training |>
# ask TA because our dataset has empty spaces instead of NA

loan_means <- loan_training |> 
    select(ApplicantIncome, CoapplicantIncome, LoanAmount, Loan_Amount_Term) |>
    map_dfr(mean, na.rm = TRUE) 
loan_means

# exploratory data analysis (plots) 

loan_distributions <- loan_training |>
    ggplot(aes(x = ApplicantIncome)) +
    geom_histogram() 
loan_distributions

ERROR: Error in loan_means: The pipe operator requires a function call as RHS (<text>:17:1)


## Methods 

#### Explain how you will conduct either your data analysis and which variables/columns you will use. 

Data analysis will be conducted using k-nearest neighbours classification through a tidymodels workflow. 
Variables we will use: 
- ApplicantIncome
- Co-applicant income
- LoanAmount
- Loan amount term 
- Credit history 
- Property area 

#### Describe at least one way that you will visualize the results

## Expected outcomes and significance

#### What do you expect to find?

Using predictors to decide if a new applicant would be eligible for a loan.

#### What impact could such findings have?

The findings could show bias in loan approval and indicate which populations may have a harder time being approved for a loan. 

#### What future questions could this lead to?

This could lead to mechanistic questions about the underlying mechanism behind whether or not someone receives a loan. One could also pose an inferential question based on the preliminary exploratory data about how the variables in this data set can be applied to the wider population. 